In [79]:
import requests
import json

import numpy as np
import pandas as pd

In [80]:
# build ULR by replacing latitude and longitude
# date of the form year_month_day e.g. 2000-01-12 --> 20000112
def build_URL(lat, lon, start_date="19810101", end_date="20201231"):

  URL = "https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?"+\
  "&request=execute&identifier=SinglePoint&"+\
  "parameters=PRECTOT,WS2M,RH2M,T2MDEW,T2M_MAX,T2M_MIN,ALLSKY_SFC_SW_DWN,T2M&"+\
  f"startDate={start_date}&endDate={end_date}&userCommunity=AG&"+\
  f"tempAverage=DAILY&outputList=CSV&lat={lat}&lon={lon}&user=DAV"

  return URL


In [81]:
build_URL(10, 12)

'https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&identifier=SinglePoint&parameters=PRECTOT,WS2M,RH2M,T2MDEW,T2M_MAX,T2M_MIN,ALLSKY_SFC_SW_DWN,T2M&startDate=19810101&endDate=20201231&userCommunity=AG&tempAverage=DAILY&outputList=CSV&lat=10&lon=12&user=DAV'

In [82]:
rq_json = requests.get(url=build_URL(10, 10)).json()

rq_json

{'features': [{'geometry': {'coordinates': [10.00001, 10.00001, 487.43],
    'type': 'Point'},
   'properties': {'parameter': {'ALLSKY_SFC_SW_DWN': {'19810101': -99.0,
      '19810102': -99.0,
      '19810103': -99.0,
      '19810104': -99.0,
      '19810105': -99.0,
      '19810106': -99.0,
      '19810107': -99.0,
      '19810108': -99.0,
      '19810109': -99.0,
      '19810110': -99.0,
      '19810111': -99.0,
      '19810112': -99.0,
      '19810113': -99.0,
      '19810114': -99.0,
      '19810115': -99.0,
      '19810116': -99.0,
      '19810117': -99.0,
      '19810118': -99.0,
      '19810119': -99.0,
      '19810120': -99.0,
      '19810121': -99.0,
      '19810122': -99.0,
      '19810123': -99.0,
      '19810124': -99.0,
      '19810125': -99.0,
      '19810126': -99.0,
      '19810127': -99.0,
      '19810128': -99.0,
      '19810129': -99.0,
      '19810130': -99.0,
      '19810131': -99.0,
      '19810201': -99.0,
      '19810202': -99.0,
      '19810203': -99.0,
      '

In [83]:
rq_json["features"][0]["properties"]['parameter'].keys()

dict_keys(['ALLSKY_SFC_SW_DWN', 'PRECTOT', 'RH2M', 'T2M', 'T2MDEW', 'T2M_MAX', 'T2M_MIN', 'WS2M'])

In [84]:
np.array(list(rq_json["features"][0]["properties"]['parameter']['T2M'].values()))

array([24.23, 22.3 , 20.86, ..., 22.98, 23.75, 24.5 ])

In [85]:
# transform date year_month_day --> date object
# date of 20200114 --> 2020-01-14
def translate_to_date(s_date):

  # split string
  year = int(s_date[0:4])
  month = int(s_date[4:6])
  day = int(s_date[6:])

  return pd.Timestamp(year=year, month=month, day=day)

# test
translate_to_date("20200114")

Timestamp('2020-01-14 00:00:00')

In [86]:
weather_dict = {}
rs = rq_json["features"][0]["properties"]['parameter']

# build a date col
# all dic obj are { "date": val }
weather_dict["date"] = [translate_to_date(d) for d in rs["PRECTOT"].keys()]

# build weather col
for key in rs:
  weather_dict[key] = list(rs[key].values())


In [87]:
weather_data_frame = pd.DataFrame(weather_dict)

In [88]:
weather_data_frame

,date,ALLSKY_SFC_SW_DWN,PRECTOT,RH2M,T2M,T2MDEW,T2M_MAX,T2M_MIN,WS2M
0,1981-01-01,-99.00,0.0,30.18,24.23,5.59,34.21,16.10,2.67
1,1981-01-02,-99.00,0.0,28.51,22.30,2.92,31.01,15.83,2.88
2,1981-01-03,-99.00,0.0,23.37,20.86,-1.09,30.17,13.26,3.51
3,1981-01-04,-99.00,0.0,26.13,20.63,0.30,29.90,12.97,3.22
4,1981-01-05,-99.00,0.0,29.72,20.40,2.06,29.16,13.01,3.25
...,...,...,...,...,...,...,...,...,...
14605,2020-12-27,19.60,0.0,46.45,21.90,9.90,30.52,14.71,2.43
14606,2020-12-28,20.08,0.0,43.04,22.20,9.00,30.84,14.77,2.52
14607,2020-12-29,20.12,0.0,44.91,22.98,10.36,31.68,15.24,2.41
14608,2020-12-30,20.11,0.0,43.38,23.75,10.53,32.98,15.43,2.28
